In [2]:
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout,Activation, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras.utils import normalize
from keras.layers import BatchNormalization,Concatenate
from keras import Input
input_shape=(2401, 1)

def load_cnn():
    
    model=Sequential()

    model.add(Conv1D(filters=8,kernel_size=5,input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=8,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=8,kernel_size=3))
    model.add(BatchNormalization())
    #model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=4))
    model.add(Conv1D(filters=16,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=16,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=16,kernel_size=3))
    model.add(BatchNormalization())
    #model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=4))
    model.add(Conv1D(filters=32,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=32,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=32,kernel_size=3))
    model.add(BatchNormalization())
    #model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=4))
    model.add(Conv1D(filters=64,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=64,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=64,kernel_size=3))
    model.add(BatchNormalization())
    #model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=4))    
    model.add(Flatten())
    model.add(Dense(128,activation='linear'))
    model.add(Dense(128,activation='linear'))
    model.add(Dense(128,activation='linear'))
    model.add(Dropout(0.5))
    model.add(Dense(2401,activation='linear'))

    from keras.optimizers import Adam
    
    learning_rate = 0.1
    decay_rate = 2
    momentum = 0.8
    opt= Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=decay_rate)
    
    model.compile(loss='mse',optimizer='adam')
    
    return model

In [3]:
input_shape=(2401, 1)
inp=Input(shape=input_shape)
convs=[]

parrallel_kernels=[3,5,7]

for k in range(len(parrallel_kernels)):
    
    conv = Conv1D(3, parrallel_kernels[k],border_mode='same',activation='linear',input_shape=input_shape,strides=1)(inp)
    convs.append(conv)

out = Concatenate()(convs)
conv_model = Model(input=inp, output=out)

model = Sequential()
model.add(conv_model)
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(128,input_dim=128,activation='relu'))
model.add(Dense(2401,input_dim=128,activation='linear'))
model.compile(loss='mse',optimizer='adam')

Instructions for updating:
Colocations handled automatically by placer.


D:\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(3, 3, activation="linear", input_shape=(2401, 1), strides=1, padding="same")`
  if __name__ == '__main__':
D:\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(3, 5, activation="linear", input_shape=(2401, 1), strides=1, padding="same")`
  if __name__ == '__main__':
D:\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(3, 7, activation="linear", input_shape=(2401, 1), strides=1, padding="same")`
  if __name__ == '__main__':
D:\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  del sys.path[0]


In [4]:
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from matplotlib import pyplot as plt

model=load_cnn()

ref_data=[]
ref_target=[]
ref_window_size=2401

count=0

data_path='data'
min_files=os.listdir(os.path.join(data_path,'MINI_BATCHES_AVG',))
for file in min_files:
    if file.endswith(".npy"):
        channel_path=os.path.join(data_path,'MINI_BATCHES_AVG',file)
        ref_dataset = np.load(channel_path)
        print(np.max(ref_dataset,axis=0))
        for i in range(ref_dataset.shape[0]-ref_window_size-1):
            ref_data.append((ref_dataset[i:i+ref_window_size,0]/6000.0).tolist())
            ref_target.append((ref_dataset[i:i+ref_window_size,1]/2500.0).tolist())
        data=np.array(ref_data)
        target=np.array(ref_target)
        
        train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)
        
        train_data=train_data.reshape(train_data.shape[0],train_data.shape[1],1)
        test_data=test_data.reshape(test_data.shape[0],test_data.shape[1],1)
        
        #target=np.array(ref_target).reshape(ref_target.shape[0],ref_target.shape[1],1)
        history=model.fit(train_data,train_target,epochs=10,validation_split=0.2)
        results=model.predict(test_data)
        print('R2_SCORE:',r2_score(test_target,results))
        print(test_target[:100])
        print(results[:100])
        
        #actual=np.argmax(results,axis=1)
        #predicted=np.argmax(target,axis=1)
        
        actual=test_target
        [0]
        predicted=results[0]
        
        actual[actual>0.05]=1
        actual[actual<0.05]=0
        
        predicted[predicted>0.05]=1
        predicted[predicted<0.05]=0        
        
        tp=actual[predicted>0]
        tp_count=len(np.where(tp>0)[0])
        fp_count=len(np.where(tp==0)[0])
                
        tn=actual[predicted==0]
        tn_count=len(np.where(tn==0)[0])
        fn_count=len(np.where(tn>0)[0])
                
        p=len(np.where(actual>0)[0])
        n=len(np.where(actual==0)[0])
                
        recall=tp_count/(tp_count+fn_count)*100
        precision=tp_count/(tp_count+fp_count)*100
        accuracy=(tp_count+tn_count)/(p+n)*100
        f1=2*(precision*recall)/(precision+recall)
                
        print("==========================window "+str(i)+"-"+str(i+ref_window_size)+"==========================")
        print("TP:",tp_count,"FP:",fp_count,"TN:",tn_count,"FN:",fn_count,"P:",p,"N:",n)
        print('Recall:',recall,"Precision:",precision,"Accuaracy:",accuracy,"F1:",f1)
        
        for i in range(len(test_target)):
            
            plt.plot(test_target[i],'b')
            plt.plot(results[i],'r')
            plt.xlabel('window')
            plt.ylabel('power')
            plt.title('MINI BATCH '+str(i)+str(count))
            plt.savefig('window_graphs_new/MINI BATCH '+str(i)+str(count)+'.png')
            plt.close()
        
        ref_data=[]
        ref_target=[]
        count=count+1

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
[1958.2275  476.    ]
Instructions for updating:
Use tf.cast instead.
Train on 1256 samples, validate on 315 samples
Epoch 1/10
1256/1256 [==============================] - 16s 13ms/step - loss: 0.0537 - val_loss: 0.0194
Epoch 2/10
1256/1256 [==============================] - 7s 5ms/step - loss: 0.0059 - val_loss: 0.0051
Epoch 3/10
1256/1256 [==============================] - 7s 5ms/step - loss: 0.0037 - val_loss: 0.0028
Epoch 4/10
1256/1256 [==============================] - 7s 5ms/step - loss: 0.0028 - val_loss: 0.0018
Epoch 5/10
1256/1256 [==============================] - 7s 5ms/step - loss: 0.0022 - val_loss: 0.0013
Epoch 6/10
1256/1256 [==============================] - 7s 5ms/step - loss: 0.0018 - val_loss: 0.0010
Epoch 7/10
1256/1256 [==============================] - 7s 5ms/step - loss: 0.0016 - val_loss: 8.3584e-04
Epoch 8/10
1256/1256 [==========================

7996/7996 [==============================] - 421s 53ms/step - loss: 4.1024e-04 - val_loss: 2.6700e-04
Epoch 2/10
7996/7996 [==============================] - 403s 50ms/step - loss: 3.9208e-04 - val_loss: 2.3967e-04
Epoch 3/10
7996/7996 [==============================] - 406s 51ms/step - loss: 3.8303e-04 - val_loss: 2.4039e-04
Epoch 4/10
7996/7996 [==============================] - 403s 50ms/step - loss: 3.7951e-04 - val_loss: 2.5165e-04
Epoch 5/10
7996/7996 [==============================] - 408s 51ms/step - loss: 3.7818e-04 - val_loss: 2.4009e-04
Epoch 6/10
7996/7996 [==============================] - 404s 51ms/step - loss: 3.7481e-04 - val_loss: 2.2732e-04
Epoch 7/10
7996/7996 [==============================] - 402s 50ms/step - loss: 3.7263e-04 - val_loss: 2.2204e-04
Epoch 8/10
7996/7996 [==============================] - 402s 50ms/step - loss: 3.7258e-04 - val_loss: 2.3269e-04
Epoch 9/10
7996/7996 [==============================] - 402s 50ms/step - loss: 3.7146e-04 - val_loss: 2.423

IndexError: tuple index out of range

In [ ]:
data_path='data'
channel_save_path=os.path.join(data_path,'T2T','ref_data.dat')
np.save(channel_save_path,np.array(ref_data))
np.savetxt(channel_save_path,np.array(ref_data))
channel_save_path=os.path.join(data_path,'T2T','ref_target.dat')
np.save(channel_save_path,np.array(ref_target))
np.savetxt(channel_save_path,np.array(ref_target))

In [ ]:
data=np.array()
data=data.reshape(data.shape[0],data.shape[1],1)
target=target.reshape(target.shape[0],data.shape[1],1)

data=data/500.0
target=target/500.0